In [1]:
import networkx as nx
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_aer import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram

import numpy as np
from scipy.optimize import minimize

In [2]:
# dij需求i到设施j的成本
# d = [[1, 2], [1, 2]]
# n = 2   # 两个设施点
# m = 2   # 两个需求点
d = [[1, 2], [3, 4], [5, 6]]
n = 2   # 两个设施点
m = 3   # 三个需求点
num_qubits = n + 2 * n * m

# gi设施i的建设成本
g = [2, 1]


In [3]:
penalty = 40
depth = 5
params = np.ones(depth * 2)

In [4]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

In [5]:
# dtype=np.complex128
def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H


def generate_Hp(n, m, d, g):
    # 初始化 Hp 矩阵为零矩阵
    # print(num_qubits)
    Hp = np.zeros((2**num_qubits, 2**num_qubits))
    for i in range(m):
        for j in range(n):
            Hp += d[i][j] * (add_in_target(num_qubits, n * (1 + i) + j) - np.eye(2**num_qubits)) / 2
    
    for j in range(n):
        Hp +=  g[j] * (add_in_target(num_qubits, j)- np.eye(2**num_qubits)) / 2

    for i in range(m):
        Ht = np.zeros((2**num_qubits, 2**num_qubits))
        for j in range(n):
            Ht += (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + i) + j)) / 2
        Ht -= np.eye(2**num_qubits)
        Hp += -penalty * Ht @ Ht
    
    for i in range(m):
        for j in range(n):
            Ht = (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + i) + j)) / 2 + (np.eye(2**num_qubits) - add_in_target(num_qubits, n * (1 + m + i) + j)) / 2 + (np.eye(2**num_qubits) - add_in_target(num_qubits, j)) / 2
            Hp += -penalty * Ht @ Ht

    return Hp

In [6]:
from scipy.linalg import expm
def build_circ(n, m, d, g, params):
  qc = QuantumCircuit(num_qubits)
  beta = params[:depth + 1]
  gamma = params[depth:]
  for i in range(num_qubits):
    qc.h(i)
  for dp in range(depth):
    qc.unitary(expm(-1j * gamma[dp] * generate_Hp(n, m, d, g)), range(num_qubits)) # transpile
    for i in range(num_qubits):
      qc.rx(beta[dp], i)
  qc.measure_all()
  return qc

In [7]:
print(build_circ(n, m, d, g, np.full(depth * 2, np.pi/4)))

         ┌───┐┌───────────┐┌─────────┐┌───────────┐┌─────────┐┌───────────┐»
    q_0: ┤ H ├┤0          ├┤ Rx(π/4) ├┤0          ├┤ Rx(π/4) ├┤0          ├»
         ├───┤│           │├─────────┤│           │├─────────┤│           │»
    q_1: ┤ H ├┤1          ├┤ Rx(π/4) ├┤1          ├┤ Rx(π/4) ├┤1          ├»
         ├───┤│           │├─────────┤│           │├─────────┤│           │»
    q_2: ┤ H ├┤2          ├┤ Rx(π/4) ├┤2          ├┤ Rx(π/4) ├┤2          ├»
         ├───┤│           │├─────────┤│           │├─────────┤│           │»
    q_3: ┤ H ├┤3          ├┤ Rx(π/4) ├┤3          ├┤ Rx(π/4) ├┤3          ├»
         ├───┤│           │├─────────┤│           │├─────────┤│           │»
    q_4: ┤ H ├┤4          ├┤ Rx(π/4) ├┤4          ├┤ Rx(π/4) ├┤4          ├»
         ├───┤│           │├─────────┤│           │├─────────┤│           │»
    q_5: ┤ H ├┤5          ├┤ Rx(π/4) ├┤5          ├┤ Rx(π/4) ├┤5          ├»
         ├───┤│           │├─────────┤│           │├─────────┤│           │»

In [8]:
def cost_function(x):
  num = [int(char) for char in x]
  C = 0
  for i in range(m):
    for j in range(n):
      C += d[i][j] * num[n * (1 + i) + j]
      
  for j in range(n):
    C += g[j] * num[j]

  for i in range(m):
    t = 0
    for j in range(n):
      t += num[n * (1 + i) + j]
    C += penalty * (t - 1)**2

  for i in range(m):
    for j in range(n):
      C += penalty * (num[n * (1 + i) + j] + num[n * (1 + m + i) + j] - num[j]) ** 2
  return C

def compute_expectation(counts):
  EV = 0
  total_count = 0
  for x, count in counts.items():
    C = cost_function(x)
    EV += C*count
    total_count += count

  return EV/total_count


def expectation_from_sample(shots = 2000):
  backend = Aer.get_backend('qasm_simulator')
  backend.shots = shots

  def execute_circ(theta):
    qc = build_circ(n, m, d, g, theta)
    counts = backend.run(qc, seed_simulator=10, shots=shots).result().get_counts()
    return compute_expectation(counts)
  
  return execute_circ

In [9]:
from numpy.lib.utils import source
from scipy.optimize import minimize
import numpy as np
# 初始化迭代计数器
iteration_count = 0
def test(pen, dep, par):
  global penalty, depth, params, iteration_count
  iteration_count = 0
  penalty = pen
  depth = dep
  params = par
  expectation = expectation_from_sample()
  def callback(x):
      global iteration_count
      iteration_count += 1
      if iteration_count % 10 == 0:
          print(f"Iteration {iteration_count}, Result: {expectation(x)}")
  # 设定最大迭代次数
  max_iterations = 1000

  # 使用 COBYLA 方法进行最小化，并设置 callback 函数
  res = minimize(expectation, params, method='COBYLA', options={'maxiter': max_iterations}, callback=callback)
  # 输出最终结果
  print("Final Result:", res)
  backend = Aer.get_backend('aer_simulator')
  backend.shots = 100000

  shots=100000
  qc_res = build_circ(n, m, d, g, params=res.x)

  counts = backend.run(qc_res, seed_simulator=10, shots = shots).result().get_counts()
  # plot_histogram(counts)
  sorted_counts = sorted(counts, key=counts.get, reverse=True)
  print("\n----------------- Full result ---------------------")
  print("selection\t\tprobability\tvalue")
  print("---------------------------------------------------")
  for x in sorted_counts[:20]:
    print(x, "{:.1f}%".format(counts[x] / shots * 100), cost_function(x))

很慢

In [10]:
test(40, 1, np.full(1 * 2, np.pi))

Iteration 10, Result: 314.8085
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 314.8085
       x: [ 3.142e+00  3.142e+00]
    nfev: 13
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
00010100000110 0.0% 286
00101100001010 0.0% 368
11011110000011 0.0% 137
11101011111000 0.0% 178
11110001110110 0.0% 212
00011110010010 0.0% 454
11100011001001 0.0% 215
11001111111000 0.0% 181
01101100011101 0.0% 329
01101011110001 0.0% 376
00110010101100 0.0% 408
10000010000001 0.0% 207
10011000011101 0.0% 407
10001100111011 0.0% 289
00100101001111 0.0% 451
11001101011010 0.0% 176
01011100010111 0.0% 250
10010011011011 0.0% 495
00100011010100 0.0% 292
00100111100100 0.0% 456


In [11]:
test(40, 2, np.full(2 * 2, np.pi))

Iteration 10, Result: 314.8085
Iteration 20, Result: 314.8085
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 314.8085
       x: [ 3.142e+00  3.142e+00  3.142e+00  3.142e+00]
    nfev: 25
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
00010100000110 0.0% 286
00101100001010 0.0% 368
11011110000011 0.0% 137
11101011111000 0.0% 178
11110001110110 0.0% 212
00011110010010 0.0% 454
11100011001001 0.0% 215
11001111111000 0.0% 181
01101100011101 0.0% 329
01101011110001 0.0% 376
00110010101100 0.0% 408
10000010000001 0.0% 207
10011000011101 0.0% 407
10001100111011 0.0% 289
00100101001111 0.0% 451
11001101011010 0.0% 176
01011100010111 0.0% 250
10010011011011 0.0% 495
00100011010100 0.0% 292
00100111100100 0.0% 456


In [12]:
test(40, 3, np.full(3 * 2, np.pi))

Iteration 10, Result: 314.8085
Iteration 20, Result: 314.8085
Iteration 30, Result: 314.8085
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 314.8085
       x: [ 3.142e+00  3.142e+00  3.142e+00  3.142e+00  3.142e+00
            3.142e+00]
    nfev: 37
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
00010100000110 0.0% 286
00101100001010 0.0% 368
11011110000011 0.0% 137
11101011111000 0.0% 178
11110001110110 0.0% 212
00011110010010 0.0% 454
11100011001001 0.0% 215
11001111111000 0.0% 181
01101100011101 0.0% 329
01101011110001 0.0% 376
00110010101100 0.0% 408
10000010000001 0.0% 207
10011000011101 0.0% 407
10001100111011 0.0% 289
00100101001111 0.0% 451
11001101011010 0.0% 176
01011100010111 0.0% 250
10010011011011 0.0% 495
00100011010100 0.0% 292
00100111100100 0.0% 456


In [13]:
test(40, 5, np.full(5 * 2, np.pi/4))

Iteration 10, Result: 274.785
Iteration 20, Result: 306.8635
